In [12]:
from MoveNet import load_model, run_movenet, process_image
import pandas as pd
import mysql.connector
import os
import numpy as np
import configparser
import mysql.connector

# MySQL Database Connect

In [13]:
config = configparser.ConfigParser()
config.read('config.ini')
db_config = config['database']

In [14]:
conn = mysql.connector.connect(
    host=db_config['host'],
    user=db_config['user'],
    password=db_config['password'],
    database=db_config['database']
)
cursor = conn.cursor()

# Pose

In [15]:
Pose_name = ['준비자세', '나란히서기', '몸통안막기', '몸통지르기', '아래막기', '앞굽이', '앞서기', '앞차기',
             '얼굴막기', '앞차고얼굴지르기', '뒷굽이', '손날바깥막기', '손날안치기', '앞차고두번지르기']

In [21]:
Pose_status = ['y']*14

In [24]:
Pose_list = list()
for x,y in zip(Pose_name,Pose_status):
    Pose_list.append((x,y))

In [26]:
query_Pose = 'INSERT INTO Pose (poseName, status) VALUES (%s,%s)'

In [27]:
cursor.executemany(query_Pose, Pose_list)

In [35]:
cursor.execute('SELECT * FROM Pose;')

In [32]:
# conn.commit()
# conn.rollback()

# Dot

In [38]:
cursor.execute('SELECT * FROM Dot;')
cursor.fetchall()

[]

In [45]:
Dot_name = ['nose', 'leftEye', 'rightEye', 'leftEar', 'rightEar', 'leftShoulder', 'rightShoulder', 'leftElbow', 'rightElbow',
            'leftWrist', 'rightWrist', 'leftHip', 'rightHip', 'leftKnee', 'rightKnee', 'leftAnkle', 'rightAnkle']

In [46]:
Dot_status = ['y']*len(Dot_name)

In [47]:
Dot_list = list()
for x,y in zip(Dot_name,Dot_status):
    Dot_list.append((x,y))

In [49]:
query_Dot = 'INSERT INTO Dot (name, status) VALUES (%s,%s)'

In [50]:
cursor.executemany(query_Dot, Dot_list)

In [52]:
conn.commit()

# Position

In [40]:
cursor.execute('SELECT * FROM Position;')
cursor.fetchall()

[(1, 1, 1, 0.5, 716.0, 249.0, 1.0, 'f', 'r', 'r', 'y', 1),
 (2, 1, 2, 0.5, 700.0, 244.0, 1.0, 'f', 'r', 'r', 'y', 1),
 (3, 1, 3, 0.5, 708.0, 245.0, 1.0, 'f', 'r', 'r', 'y', 1),
 (4, 1, 4, 0.5, 677.0, 248.0, 1.0, 'f', 'r', 'r', 'y', 1),
 (5, 1, 5, 0.5, 685.0, 257.0, 1.0, 'f', 'r', 'r', 'y', 1),
 (6, 1, 6, 0.5, 714.0, 296.0, 1.0, 'f', 'r', 'r', 'y', 1),
 (7, 1, 7, 0.5, 648.0, 315.0, 1.0, 'f', 'r', 'r', 'y', 1),
 (8, 1, 8, 0.5, 725.0, 337.0, 1.0, 'f', 'r', 'r', 'y', 1),
 (9, 1, 9, 0.5, 675.0, 362.0, 1.0, 'f', 'r', 'r', 'y', 1),
 (10, 1, 10, 0.5, 722.0, 400.0, 1.0, 'f', 'r', 'r', 'y', 1),
 (11, 1, 11, 0.5, 763.0, 353.0, 1.0, 'f', 'r', 'r', 'y', 1),
 (12, 1, 12, 0.5, 717.0, 409.0, 1.0, 'f', 'r', 'r', 'y', 1),
 (13, 1, 13, 0.5, 684.0, 415.0, 1.0, 'f', 'r', 'r', 'y', 1),
 (14, 1, 14, 0.5, 756.0, 448.0, 1.0, 'f', 'r', 'r', 'y', 1),
 (15, 1, 15, 0.5, 706.0, 502.0, 1.0, 'f', 'r', 'r', 'y', 1),
 (16, 1, 16, 0.5, 792.0, 493.0, 1.0, 'f', 'r', 'r', 'y', 1),
 (17, 1, 17, 0.5, 693.0, 556.0, 1.0, 'f', 

In [56]:
model = load_model()

In [192]:
# 수시로 변경
hand = ['r']*17
foot = ['r']*17
file = '태극3장_앞차고두번지르기_왼쪽_5'

# 고정
file_path = "C:\\Users\\wlghks\\Desktop\\24-1\\Taekwondo\\Taekwondo_1,2,3장_기본동작_편집본\\단위동작\\"+file+".jpg"
file_image = process_image(file_path)

result = run_movenet(model, file_image)
result = result.reshape(-1,3)

file_name = os.path.basename(file_path)

file_name_split = file_name.replace('.jpg','').split('_')
Poomsae, pose, dirt, pSeq = file_name_split

Pose_name = ['준비자세', '나란히서기', '몸통안막기', '몸통지르기', '아래막기', '앞굽이', '앞서기', '앞차기',
             '얼굴막기', '앞차고얼굴지르기', '뒷굽이', '손날바깥막기', '손날안치기', '앞차고두번지르기']

# 고정
poseIdx = [Pose_name.index(pose) + 1]*len(result)

dotIdx = list(range(1,18))

reliability = [i[2] for i in result]
reliability = [float(i) if isinstance(i, np.float32) else i for i in reliability]

x = [i[0] for i in result]
x = [float(i) if isinstance(i, np.float32) else i for i in x]

y = [i[1] for i in result]
y = [float(i) if isinstance(i, np.float32) else i for i in y]

z = [1.0] * len(result)

match dirt:
    case '정면':
        direction = ['f']*len(result)
    case '왼쪽':
        direction = ['l']*len(result)
    case '오른쪽':
        direction = ['r']*len(result)

status = ['y']*len(result)

poseSeq = [int(pSeq)]*len(result)

Position_list_T = [poseIdx, dotIdx, reliability, x, y, z, direction, hand, foot, status, poseSeq]

Position_list = list(map(list, zip(*Position_list_T)))

query_Dot = '''INSERT INTO Position (poseIdx, dotIdx, reliability, x, y, z, direction, hand, foot, status, poseSeq)
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)'''

cursor.executemany(query_Dot, Position_list)

In [194]:
conn.commit()

In [ ]:
# conn.rollback()

In [195]:
conn.close()

# Poomsae

In [39]:
cursor.execute('SELECT * FROM Poomsae;')
cursor.fetchall()

[]

In [53]:
Poomsae_num = list(range(1,4))

In [ ]:
# poseOrder = '1,2,3, ...'

In [54]:
Poomsae_status = ['y']*len(Poomsae_num)